# Ottenimento prossimi match
Lo script deve permettere di prendere i prossimi 10 match. 
Verrà fatta una predizione su questi match, prima deve essere calcolata la media di ciascun match che è data dai 5 match precedenti.

In [47]:
from analysis import MatchAnalysis
import pandas as pd
import util_strings as utils
from end.ranking import Ranking
from football_predictions import FootballPredictions

In [48]:
championship = pd.read_csv(utils.championship_actual_season, index_col=0)
championship['date'] = pd.to_datetime(championship['date'], format='%Y-%m-%d')

championship.result = championship.result.astype(str)
new_matches = championship[championship.result == 'nan']

In [49]:
matches = new_matches[:10][['team1', 'team2', 'date', 'result']]
matches.rename(columns= {'team1': 'home', 'team2': 'away'}, inplace=True)
matches.reset_index(drop=True, inplace=True)

## Ottenimento testo FootballPrediction
Scaricamento del testo di ciascuna partita

In [50]:
fp = FootballPredictions(matches=matches)
fp.update_originalNotation()
fp.get_urls()
fp.get_predictions(utils.next_matches)
fp.recovery_games()
fp.matches_not_found()
fp.fix_dates(utils.next_matches)

100%|██████████| 4/4 [00:02<00:00,  1.66it/s]


In [51]:
from my_tokenizer import MyTokenizer

mt = MyTokenizer(fp.df)
mt.clean_text()
mt.open_vectorizer(utils.vectorizer)

In [52]:
import pickle

dt = pickle.load(open('ML model/ta decision tree 0.942 (no syn).model', 'rb'))
predictions = dt.predict(mt.vectorizer.transform(mt.cleaned_corpus))

In [53]:
cont = 0
for i, el in fp.df.iterrows():
    if (el.prediction == predictions[i]):
        cont += 1
          
print(cont)

7


Aggiungo le predizioni al dataset dei match

In [54]:
cont = 0
for i, match in matches.iterrows():
    matches.at[i, 'prediction'] = predictions[cont]
    cont += 1

## Calcolo delle medie
Per ciascun match devo calcolare le statistiche date dalle medie dei 5 match precedenti e poi devo ottenere il testo da football_prediction

In [55]:
ranking = Ranking('SerieA', '2022-2023')
ranking.read_matches(seasons=4, path=utils.ranking)

ma = MatchAnalysis()
ma.set_ranking(ranking=ranking)
#leggo tutti i match (doppi perché ci sono le statistiche x ogni squadra)
ma.read_matches(utils.merged_statistics)
#creo una lista di oggetti (lista di squadre)
ma.create_team_dataset() 

matches['date'] = pd.to_datetime(matches['date'], format='%Y-%m-%d') 
float_features_and_avg = ['goals','total_shots','shots_on_target','goals_on_penalty','total_penalties','percentage_possession','fouls','yellow_cards','red_cards']

import numpy as np
for f in float_features_and_avg:
    matches[f] = np.nan

matches[float_features_and_avg] = matches[float_features_and_avg].astype(float)

label = {'N': 0, 'V': 1, 'P':2}

ma.diff_dataset = matches.copy()
for i, match in ma.diff_dataset.iterrows(): 
    match.season = '2022-2023'
    avgH, c = ma.get_team_by_name(match.home).get_avg_last_X_matches(5, match.date, float_features_and_avg)
    avgA, c = ma.get_team_by_name(match.away).get_avg_last_X_matches(5, match.date, float_features_and_avg)
    
    ma.get_ranks(match, i)

    for col in float_features_and_avg:
        diff = avgH[col] - avgA[col]
        ma.diff_dataset.at[i, col] = diff
    
    ma.diff_dataset.at[i, 'prediction'] = label[match.prediction]

    ma.diff_dataset.at[i, 'home'] = ma.get_id_by_name(match.home)
    ma.diff_dataset.at[i, 'away'] = ma.get_id_by_name(match.away)

ma.diff_dataset.drop(columns=['result', 'date'], inplace=True)

In [56]:
ma.diff_dataset
dt = pickle.load(open('ML model/random forest 0.549.model', 'rb'))
predictions = dt.predict(ma.diff_dataset)

In [58]:
df_predictions = pd.DataFrame()
for i, match in matches.iterrows():
    match['prediction'] = predictions[i]
    df_predictions = df_predictions.append(match[['home','away','prediction']])

In [59]:
df_predictions

,home,away,date,result,prediction,goals,total_shots,shots_on_target,goals_on_penalty,total_penalties,percentage_possession,fouls,yellow_cards,red_cards
0,Juventus,Empoli,2022-10-21,nan,V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Salernitana,Spezia,2022-10-22,nan,V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fiorentina,Inter,2022-10-22,nan,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Milan,Monza,2022-10-22,nan,V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Atalanta,Lazio,2022-10-23,nan,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Bologna,Lecce,2022-10-23,nan,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Roma,Napoli,2022-10-23,nan,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Udinese,Torino,2022-10-23,nan,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Sassuolo,Hellas Verona,2022-10-24,nan,P,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Cremonese,Sampdoria,2022-10-24,nan,V,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
